In [221]:
import pandas as pd
import graphviz
import os
from pyagrum_extra import gum
from pyagrum_extra import predict
import pyAgrum.lib.notebook as gnb 


ot_odr_filename = os.path.join(".", "donnees/OT_ODR.csv.bz2")
ot_odr_df = pd.read_csv(ot_odr_filename,
                        compression="bz2",
                        sep=";")

equipements_filename = os.path.join(".", 'donnees/EQUIPEMENTS.csv')
equipements_df = pd.read_csv(equipements_filename,
                             sep=";")

# on fait un seul tableau à partir de nos 2 csv
merged_df = pd.merge(ot_odr_df, equipements_df, on='EQU_ID')

#on supprime une ligne qui possède des valeurs non définit (kilométrage)
merged_df.drop(178662, inplace=True)

# Définition des limites des intervalles pour chaque classe
kilometrage_bins = [-1, 150000, 300000,float('inf')]

# Définition des étiquettes pour chaque classe
kilometrage_labels = ['0-150000', '150000-300000','300000+']

# Transformation de la variable "kilométrage" en classe
merged_df['KILOMETRAGE_CLASSE'] = pd.cut(merged_df['KILOMETRAGE'], bins=kilometrage_bins, labels=kilometrage_labels)
merged_df = merged_df.sort_values('KILOMETRAGE')
print(merged_df["KILOMETRAGE_CLASSE"])
print(merged_df["KILOMETRAGE"])

# Conversion de la nouvelle variable en catégorie
merged_df['KILOMETRAGE_CLASSE'] = merged_df['KILOMETRAGE_CLASSE'].astype('category')

var_sig = ["SIG_ORGANE", "SIG_CONTEXTE", "SIG_OBS"]
merged_df[var_sig].describe()

var_sys = ["SYSTEM_N1", "SYSTEM_N2", "SYSTEM_N3"]
merged_df[var_sys].describe()

var_odr = ["TYPE_TRAVAIL", "ODR_LIBELLE"]
merged_df[var_odr].describe()

var_cat = ['ODR_LIBELLE', 'TYPE_TRAVAIL',
           'SYSTEM_N1', 'SYSTEM_N2', 'SYSTEM_N3', 
           'SIG_ORGANE', 'SIG_CONTEXTE', 'SIG_OBS', 'LIGNE']
for var in var_cat:
    merged_df[var] = merged_df[var].astype('category')

merged_df.info()

var_to_model = ["SYSTEM_N1", "SIG_OBS","SIG_ORGANE","KILOMETRAGE_CLASSE"]

var_bn = {}
for var in var_to_model:
    nb_values = len(merged_df[var].cat.categories)
    var_bn[var] = gum.LabelizedVariable(var, var, nb_values)
    
for var in var_bn:
    for i, modalite in enumerate(merged_df[var].cat.categories):
        var_bn[var].changeLabel(i, modalite)
        




505655    0-150000
433726    0-150000
439684    0-150000
439685    0-150000
497676    0-150000
            ...   
10494      300000+
10495      300000+
10496      300000+
10497      300000+
10498      300000+
Name: KILOMETRAGE_CLASSE, Length: 506557, dtype: category
Categories (3, object): ['0-150000' < '150000-300000' < '300000+']
505655    0.000000e+00
433726    0.000000e+00
439684    0.000000e+00
439685    0.000000e+00
497676    0.000000e+00
              ...     
10494     3.154686e+06
10495     3.155818e+06
10496     3.155818e+06
10497     3.157933e+06
10498     3.159324e+06
Name: KILOMETRAGE, Length: 506557, dtype: float64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 506557 entries, 505655 to 10498
Data columns (total 19 columns):
 #   Column              Non-Null Count   Dtype   
---  ------              --------------   -----   
 0   OT_ID               506557 non-null  object  
 1   ODR_ID              506557 non-null  object  
 2   ODR_LIBELLE         506557 non-null  ca

In [222]:
bn = gum.BayesNet("modèle simple")
for var in var_bn.values():
    bn.add(var)
    
bn.addArc("SIG_OBS", "SYSTEM_N1")
bn.addArc("SIG_ORGANE", "SYSTEM_N1")
bn.addArc("KILOMETRAGE_CLASSE", "SYSTEM_N1")



#vérification de notre RB
import pyAgrum.lib.notebook as gnb 
bn

# bn.cpt("SYSTEM_N1")

(pyAgrum.BayesNet<double>@00000265840FA410) BN{nodes: 4, arcs: 3, domainSize: 205320, dim: 184963, mem: 1Mo 581Ko 464o}

In [223]:
# bn.cpt("SIG_OBS")

In [224]:
bn.fit_bis(merged_df, verbose_mode=True)
# bn.cpt("SIG_OBS")


- Learn CPT SIG_OBS
- Learn CPT SYSTEM_N1
- Learn CPT SIG_ORGANE
- Learn CPT KILOMETRAGE_CLASSE


In [225]:
# bn.cpt("SYSTEM_N1")



In [226]:
pred_prob = bn.predict_proba(merged_df[["SIG_OBS","SIG_ORGANE","KILOMETRAGE_CLASSE"]].iloc[-1000:], 
                             var_target="SYSTEM_N1",
                             show_progress=True)





In [227]:
print(pred_prob)



[[0.00000000e+00 1.09375000e-01 0.00000000e+00 ... 3.12500000e-02
  0.00000000e+00 1.56250000e-02]
 [0.00000000e+00 1.09375000e-01 0.00000000e+00 ... 3.12500000e-02
  0.00000000e+00 1.56250000e-02]
 [0.00000000e+00 1.09375000e-01 0.00000000e+00 ... 3.12500000e-02
  0.00000000e+00 1.56250000e-02]
 ...
 [0.00000000e+00 1.69405387e-04 1.69405387e-04 ... 9.95849568e-01
  6.49387317e-04 5.64684624e-05]
 [0.00000000e+00 1.72473267e-04 2.58709900e-04 ... 9.95343222e-01
  4.31183167e-04 0.00000000e+00]
 [0.00000000e+00 4.11285679e-04 2.46771407e-04 ... 9.62902032e-01
  3.29028543e-04 1.64514272e-04]]


In [228]:
pred = bn.predict(merged_df[["SIG_OBS","SIG_ORGANE","KILOMETRAGE_CLASSE"]].iloc[-1000:], 
                  var_target="SYSTEM_N1",
                  show_progress=True)
print(pred)

['EQUIPEMENT DE FREINAGE' 'EQUIPEMENT DE FREINAGE'
 'EQUIPEMENT DE FREINAGE' 'EQUIPEMENT DE FREINAGE'
 'EQUIPEMENT DE FREINAGE' 'EQUIPEMENT DE FREINAGE'
 'EQUIPEMENT PNEUMATIQUE' 'EQUIPEMENT ELECTRIQUE'
 'EQUIPEMENT DE MOTORISATION' 'EQUIPEMENT DE MOTORISATION'
 'EQUIPEMENT DE MOTORISATION' 'EQUIPEMENT DE CARROSSERIE'
 'EQUIPEMENT ELECTRIQUE' 'EQUIPEMENT ELECTRIQUE' 'EQUIPEMENT ELECTRIQUE'
 'EQUIPEMENT ELECTRIQUE' 'EQUIPEMENT ELECTRIQUE'
 'EQUIPEMENT DE CARROSSERIE' 'EQUIPEMENT ELECTRIQUE'
 'EQUIPEMENT DE MOTORISATION' 'EQUIPEMENT DE MOTORISATION'
 'EQUIPEMENT ELECTRIQUE' 'EQUIPEMENT ELECTRIQUE'
 'EQUIPEMENT DE MOTORISATION' 'EQUIPEMENT DE MOTORISATION'
 'EQUIPEMENT DE CARROSSERIE' 'EQUIPEMENT DE CARROSSERIE'
 'EQUIPEMENT DE CARROSSERIE' 'EQUIPEMENT DE CARROSSERIE'
 'EQUIPEMENT DE FREINAGE' 'EQUIPEMENT ELECTRIQUE'
 'EQUIPEMENT DE CARROSSERIE' 'EQUIPEMENT EMBARQUE'
 'EQUIPEMENT DE CARROSSERIE' 'EQUIPEMENT ELECTRIQUE'
 'EQUIPEMENT ELECTRIQUE' 'EQUIPEMENT ELECTRIQUE'
 'EQUIPEMENT DE CARRO

In [241]:
accuracy = bn.predict(merged_df[["SIG_OBS","SIG_ORGANE","KILOMETRAGE_CLASSE"]].iloc[-5000:],var_target="SYSTEM_N1",show_progress=True)
print(accuracy)

['EQUIPEMENT DE CARROSSERIE' 'EQUIPEMENT DE CARROSSERIE'
 'EQUIPEMENT ELECTRIQUE' ... 'EQUIPEMENT ELECTRIQUE'
 'EQUIPEMENT ELECTRIQUE' 'EQUIPEMENT ELECTRIQUE']


In [238]:
merged_df

,OT_ID,ODR_ID,ODR_LIBELLE,TYPE_TRAVAIL,DUREE_TRAVAIL,SYSTEM_N1,SYSTEM_N2,SYSTEM_N3,EQU_ID,DATE_OT,KILOMETRAGE,SIG_ORGANE,SIG_CONTEXTE,SIG_OBS,LIGNE,MODELE,CONSTRUCTEUR,MOTEUR,KILOMETRAGE_CLASSE
505655,OT000326415,OM000404264,REMPLACEMENT DU BOITIER TECHNIQUE SAM,EQUIPEMENT EMBARQUE,0.08,EQUIPEMENT EMBARQUE,AIDE A L'EXPLOITATION-MAINTENANCE,BOITIER TECHNIQUE SAM,E00340050,2018-09-04 17:56:07,0.000000e+00,RADIO,POSTE CONDUITE,NE FONCTIONNE PAS,L0012,MD062,C007,MT023,0-150000
433726,OT000031333,OM000036159,REMPLACEMENT D'UN ECLAIRAGE EXT,MECANIQUE,0.25,EQUIPEMENT ELECTRIQUE,ECLAIRAGE-SIGNALISATION,ECLAIRAGE-SIGNALISATION EXT,E00070292,2014-04-10 21:50:55,0.000000e+00,ECLAIRAGE FEUX EXTERIEURS,AVANT/GAUCHE,NE FONCTIONNE PAS,L0489,MD030,C006,MT027,0-150000
439684,OT000038308,OM000045741,REMPLACEMENT D'UNE COURROIE D'ALTERNATEUR,MECANIQUE,0.33,EQUIPEMENT ELECTRIQUE,CIRCUIT DE CHARGE,ORGANE DU CIRCUIT DE CHARGE,E00073750,2014-05-30 19:03:18,0.000000e+00,MOTEUR,AU POINT MORT/ARRIERE,BRUYANT,L0258,MD046,C008,MT029,0-150000
439685,OT000053873,OM000065763,REMPLACEMENT DU BOITIER PUPITRE ICS,EQUIPEMENT EMBARQUE,0.08,EQUIPEMENT EMBARQUE,AIDE A L'EXPLOITATION-MAINTENANCE,PUPITRE ICS,E00073750,2014-09-22 19:11:40,0.000000e+00,ANNONCEUR SONORE,INTERIEUR/EXTERIEUR,NE FONCTIONNE PAS,L0258,MD046,C008,MT029,0-150000
497676,OT000205971,OM000251438,REMPLACEMENT D'UN BALAI D'ESSUIE-GLACE,MECANIQUE,0.16,EQUIPEMENT DE CARROSSERIE,ACCESSOIRE DE VISIBILITE,ESSUIE-GLACE,E00302275,2017-01-10 21:33:15,0.000000e+00,ESSUIE-GLACE,AVANT,ARRACHE,L0124,MD033,C006,MT022,0-150000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10494,OT000019365,OM000021853,REMPLACEMENT D'UNE LAMPE DE CODE / PHARE,ELECTRICITE,0.04,EQUIPEMENT ELECTRIQUE,ECLAIRAGE-SIGNALISATION,ECLAIRAGE-SIGNALISATION EXT,E00004844,2014-01-24 19:05:37,3.154686e+06,ECLAIRAGE FEUX EXTERIEURS,DROIT/AVANT,NE FONCTIONNE PAS,L0256,MD045,C008,MT033,300000+
10495,OT000021641,OM000024406,REMPLACEMENT D'UNE LAMPE DE FEU CLIGNOTANT,ELECTRICITE,0.04,EQUIPEMENT ELECTRIQUE,ECLAIRAGE-SIGNALISATION,ECLAIRAGE-SIGNALISATION EXT,E00004844,2014-02-06 23:15:18,3.155818e+06,CLIGNOTANTS/ DETRESSE,LATERAL/DROIT/ARRIERE,ABSENT,L0256,MD045,C008,MT033,300000+
10496,OT000021642,OM000024405,REMPLACEMENT D'UNE LAMPE DE FEU STOP,ELECTRICITE,0.04,EQUIPEMENT ELECTRIQUE,ECLAIRAGE-SIGNALISATION,ECLAIRAGE-SIGNALISATION EXT,E00004844,2014-02-06 23:15:27,3.155818e+06,ECLAIRAGE FEUX EXTERIEURS,EXTERIEUR/ARRIERE/GAUCHE,NE FONCTIONNE PAS,L0256,MD045,C008,MT033,300000+
10497,OT000023623,OM000026980,REMPLACEMENT D'UNE LAMPE DE FEU STOP,ELECTRICITE,0.04,EQUIPEMENT ELECTRIQUE,ECLAIRAGE-SIGNALISATION,ECLAIRAGE-SIGNALISATION EXT,E00004844,2014-02-21 20:41:49,3.157933e+06,ECLAIRAGE FEUX EXTERIEURS,CENTRE/CENTRALE/BAS/DROIT,ABSENT,L0256,MD045,C008,MT033,300000+
